##### Прогресс

* `[v]` Упражнение 3-4  - удаление бесполезных символов
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики


* `[ ]` Упражнение 5-6  - преобразование к неукорачивающей грамматике
    * ???


* `[v]` Упражнение 7-8  - удаление цепных правил
    * https://neerc.ifmo.ru/wiki/index.php?title=Удаление_цепных_правил_из_грамматики


* `[ ]` Упражнение 9-10 - исключение левой рекурсии из свободной грамматики
    * `[v]` https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики
    * `[ ]` https://neerc.ifmo.ru/wiki/index.php?title=Устранение_левой_рекурсии


* `[ ]` Упражнение 11-12 - приведение к нормальной форме Хомского
    * `[v]` https://neerc.ifmo.ru/wiki/index.php?title=Удаление_длинных_правил_из_грамматики
    * `[v]` https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики
    * `[v]` https://neerc.ifmo.ru/wiki/index.php?title=Удаление_цепных_правил_из_грамматики 
    * `[v]` https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики
    * `[ ]` https://neerc.ifmo.ru/wiki/index.php?title=Нормальная_форма_Хомского

In [37]:
import re

import itertools as it

In [38]:
def union_lists(a, b):
    return list(set(a).union(set(b)))

def is_subset_of(a, b):
    return [x for x in a if x in b] == a

def intersection(a, b):
    return [x for x in a if x in b]

def has_intersection(a, b):
    return len(intersection(a, b)) > 0

In [39]:
def left(r):
    return r.split('->')[0].strip()

def right(r):
    return r.split('->')[1].strip()

In [40]:
NON_TERMINAL = r'[A-Z][\d]*'
TERMINAL     = r'[^A-Z\d\s\->\|]'

def all_terminals(g):
    return set([q for x in g for q in re.findall(TERMINAL, x)])

def all_non_terminals(g):
    return set([q for x in g for q in re.findall(NON_TERMINAL, x)])

def _non_terminals(r, right=True):
    i = int(right)
    return [x for x in re.findall(NON_TERMINAL, r.split('->')[i])]

def _terminals(r, right=True):
    i = int(right)
    return [x for x in re.findall(TERMINAL, r.split('->')[i])]

def right_non_terminals(r):
    return _non_terminals(r)

def right_terminals(r):
    return _terminals(r)

def left_non_terminals(r):
    return _non_terminals(r, right=False)

def left_terminals(r):
    return _terminals(r, right=False)

In [41]:
def explode(r):
    left, right = r.split('->')
    right = right.split('|')
    return [(left + '->' + p).strip() \
               for p in right]

def explode_grammar(g):
    return [x for r in g for x in explode(r)]

def compact_grammar(g):
    from collections import defaultdict
    dict_ = defaultdict(list)
    for r in g:
        left, right = r.split('->')
        left, right = left.strip(), right.strip()
        dict_[left] += [right]
    return [k + ' -> ' + ' | '.join(v) for k,v in dict_.items()]

# Удаление непорождающих правил

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики

In [46]:
g = [
    'S -> AS | BS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [47]:
def delete_non_generating_rules(g):
    g_ = explode_grammar(g)
    Q = []
    for r in g_:
        if len(right_non_terminals(r)) == 0:
            Q = union_lists(Q, left_non_terminals(r)) 
    changed = True
    while changed:
        changed = False
        for r in g_:
            if is_subset_of(right_non_terminals(r), Q): 
                old = len(Q)
                Q = union_lists(Q, left_non_terminals(r))
                if old != len(Q):
                    changed = True
    return compact_grammar([r for r in g_ if is_subset_of(right_non_terminals(r), Q)])

In [48]:
new_grammar = delete_non_generating_rules(g)

In [49]:
for r in new_grammar:
    print(r)

S -> AS | s
E -> EF | FF
A -> a
F -> f


# Удаление недостижимых нетерминалов

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики

Нетерминал $A$ называется достижимым (англ. reachable) в КС-грамматике $\Gamma$, если существует порождение $S \Rightarrow^* \alpha A \beta$. Иначе он называется недостижимым (англ. unreachable).

Очевидно, что если нетерминал в левой части правила является достижимым, то и все нетерминалы правой части являются достижимыми.

Алгоритм

* __Шаг 0.__ Множество достижимых нетерминалов состоит из единственного элемента: $\lbrace S \rbrace$.
* __Шаг 1.__ Если найдено правило, в левой части которого стоит нетерминал, содержащийся в множестве, добавим в множество все нетерминалы из правой части.
* __Шаг 2.__ Повторим предыдущий шаг, если множество порождающих нетерминалов изменилось.

Получаем множество всех достижимых нетерминалов, а нетерминалы, не попавшие в него, являются недостижимыми.

In [50]:
g = [
    'S -> AS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [51]:
def delete_unreachable_non_terminals(g):
    g_ = explode_grammar(g)
    Q = ['S']
    changed = True
    while changed:
        changed = False
        for r in g_:
            lnt = left_non_terminals(r).pop()
            if lnt not in Q:
                continue
            old = len(Q)
            Q = union_lists(Q, right_non_terminals(r))
            if old != len(Q):
                changed = True
    return compact_grammar([r for r in g_ if left_non_terminals(r).pop() in Q])

In [52]:
new_grammar = delete_unreachable_non_terminals(g)

In [53]:
for r in new_grammar:
    print(r)

S -> AS | s
A -> a


# Удаление бесполезных символов

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_бесполезных_символов_из_грамматики

Нетерминал $A$ называется полезным (англ. useful) в КС-грамматике $\Gamma$, если он может участвовать в выводе, то есть существует порождение вида $S \Rightarrow ^* \alpha A \beta \Rightarrow ^* w$. Иначе он называется бесполезным (англ. useless).

Грамматика $\Gamma$ не содержит бесполезных нетерминалов тогда и только тогда, когда грамматика $\Gamma$ не содержит ни недостижимых нетерминалов, ни непорождающих.

Алгоритм состоит из двух этапов:

1. Удалить из грамматики правила, содержащие непорождающие нетерминалы.
2. Удалить из грамматики правила, содержащие недостижимые нетерминалы.

In [54]:
def delete_useless_symbols(g):
    g_ = delete_non_generating_rules(g)
    return delete_unreachable_non_terminals(g_)

In [55]:
g = [
    'S -> AS | BS | s',
    'E -> EF | FF',
    'A -> a',
    'F -> f'
]

In [56]:
new_grammar = delete_useless_symbols(g)

In [57]:
for r in new_grammar:
    print(r)

S -> AS | s
A -> a


# Удаление цепных правил

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_цепных_правил_из_грамматики

__Цепная пара__ (unit pair) — упорядоченная пара (A,B), в которой $A\Rightarrow ^* B$, используя только цепные правила.

Алгоритм удаления цепных правил из грамматики:

1. Найти все цепные пары в грамматике $\Gamma$.
2. Для каждой цепной пары $(A,B)$ добавить в грамматику $\Gamma'$ все правила вида $A\rightarrow\alpha$, где $B\rightarrow\alpha$ — нецепное правило из $\Gamma$.
3. Удалить все цепные правила

In [58]:
g = [
    'A -> B | a', 
    'B -> C | b',
    'C -> DD | c'
] 

In [59]:
def delete_unit_rules(g):
    g_ = explode_grammar(g)
    pairs = [(k, k) for k in all_non_terminals(g)]
    
    to_delete = []
    for r in g_:
        lnt = left_non_terminals(r).pop()
        rnt = right_non_terminals(r)
        if len(rnt) > 1:
            continue
        for a, b in pairs:  
            if b == lnt:
                for x in rnt:
                    if (a, x) not in pairs:
                        to_delete += [r]
                        pairs += [(a, x)]

    pairs = [(a,b) for (a,b) in pairs if a != b] 
    
    new_rules = []
    for a,b in pairs:
        for r in g_:
            lnt = left_non_terminals(r).pop()
            if lnt != b:
                continue
            candidate = (a, right(r))
            if candidate not in pairs:
                new_rules += [a + ' -> ' + right(r)]
    
    new_rules.extend(g_)
    new_rules = [x for x in new_rules if x not in to_delete]
    
    return compact_grammar(new_rules)

In [60]:
new_grammar = delete_unit_rules(g)

In [61]:
for r in new_grammar:
    print(r)

A -> b | DD | c | a
B -> DD | c | b
C -> DD | c


# Поиск $\varepsilon$-нетерминалов

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики

Правила вида $A \to \varepsilon$ называются $\varepsilon$-правилами (англ. $\varepsilon$-rule).

Нетерминал $A$ называется $\varepsilon$-порождающим (англ. $\varepsilon$-generating), если $A \Rightarrow^* \varepsilon$.

Алгоритм поиска $\varepsilon$-порождающих нетерминалов

1. Найти все $\varepsilon$-правила. Составить множество, состоящее из нетерминалов, входящих в левые части таких правил.
2. Перебираем правила грамматики $\Gamma$. Если найдено правило $A \rightarrow C_1C_2...C_k$, для которого верно, что каждый $C_i$ принадлежит множеству, то добавить $A$ в множество.
3. Если на шаге 2 множество изменилось, то повторить шаг 2.

__NB:__ Будем обозначать $\varepsilon$ в коде как #.

In [62]:
g = [
    'S -> ABC',
    'S -> DS',
    'A -> #',
    'B -> AC',
    'C -> #',
    'D -> d'
]

In [63]:
def find_esp_non_terminals(g):
    g_ = explode_grammar(g)
    Q = []
    changed = True
    while changed:
        changed = False
        for r in g_:
            all_non_terms_are_eps = is_subset_of(right_non_terminals(r), Q) and len(right_terminals(r)) == 0
            if right(r) == '#' or all_non_terms_are_eps:
                old = len(Q)
                Q = union_lists(Q, left_non_terminals(r))
                if old != len(Q):
                    changed = True
    return Q

In [64]:
Q = find_esp_non_terminals(g)

In [65]:
print(Q)

['C', 'B', 'A', 'S']


# Удаление $\varepsilon$-правил

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_eps-правил_из_грамматики

Алгоритм удаления $\varepsilon$-правил из грамматики

1. Добавить все правила из $P$ в $P'$.
2. Найти все $\varepsilon$-порождаюшие нетерминалы.
3. Для каждого правила вида $A \rightarrow \alpha_0 B_1 \alpha_1 B_2 \alpha_2 ... B_k \alpha_k$ (где $\alpha_i$ — последовательности из терминалов и нетерминалов, $B_j$ — $\varepsilon$-порождающие нетерминалы) добавить в $P'$ все возможные варианты правил, в которых либо присутствует, либо удалён каждый из нетерминалов $B_j\; (1 \leqslant j \leqslant k)$.
4. Удалить все $\varepsilon$-правила из $P'$.
5. Если в исходной грамматике $\Gamma$ выводилось $\varepsilon$, то необходимо добавить новый нетерминал $S'$, сделать его стартовым, добавить правило $S' \rightarrow S|\varepsilon$.

In [66]:
g = [
    'S -> ABCd',
    'A -> a | #',
    'B -> AC',
    'C -> c | #'
]

In [67]:
def delete_eps_generating_rules(g):
    all_combinations = lambda s: [x for y in [list(it.combinations([x for x in s], i+1)) \
                                      for i in range(len(s))] for x in y] 
    
    g_ = explode_grammar(g)
    
    new_rules = []
    eps_gen = find_esp_non_terminals(g)
    
    for r in g_:
        if has_intersection(right_non_terminals(r), eps_gen):
            to_add = []
            s = right(r)
            for t in all_combinations(''.join(intersection(eps_gen, right_non_terminals(r)))):
                s_ = s[:]
                for c in t:
                    s_ = s_.replace(c, '')
                if len(s_.strip()) > 0:
                    to_add += [left(r) + ' -> ' + s_]
            new_rules.extend(to_add)

    g_ = [x for x in g_ if right(x) != '#']
    g_.extend(new_rules)
    if 'S' in eps_gen:
        g_ += ['S -> #']
    return compact_grammar(g_)

In [68]:
for r in delete_eps_generating_rules(g):
    print(r)

S -> ABCd | BCd | ABd | ACd | Bd | Cd | Ad | d
A -> a
B -> AC | C | A
C -> c


# Удаление длинных правил из грамматики

https://neerc.ifmo.ru/wiki/index.php?title=Удаление_длинных_правил_из_грамматики

Пусть  $\Gamma$ — контекстно-свободная грамматика. Правило $A \rightarrow \beta$ называется __длинным__ (англ. long rule), если $|\beta| > 2$.

Пусть $\Gamma$ — контекстно-свободная грамматика, содержащая длинные правила. Требуется построить эквивалентную грамматику $\Gamma'$, не содержащую длинных правил. 
Задача удаления длинных правил из грамматики возникает при попытке её приведения к нормальной форме Хомского.

С каждым длинным правилом $A \rightarrow a_1 a_2 \ldots a_k$, $k > 2$, $a_i \in \Sigma \cup N$ проделаем следующее:

1. Добавим в грамматику $k-2$ новых нетерминала $B_1, B_2, \ldots B_{k-2}$.
2. Добавим в грамматику $k-1$ новое правило:
    $$
    A \rightarrow a_1B_1 \\
    B_1 \rightarrow a_2B_2 \\
    B_2 \rightarrow a_3B_3 \\
    \ldots \\
    B_{k-2} \rightarrow a_{k-1}a_{k} 
    $$
3. Удалим из грамматики правило $A \rightarrow a_1 a_2 \ldots a_k$.

In [69]:
g = [
    'S -> AB',
    'A -> aBcB',
    'B -> def'
]

In [70]:
def delete_long_rules(g):
    split_right = lambda s: [s[i:i+1+(1 if i == len(s)-2 else 0)] for i in range(len(s)-1)]

    g_ = explode_grammar(g)
    new_grammar = []
    for r in g_:
        if len(right_non_terminals(r)) + len(right_terminals(r)) <= 2:
            new_grammar += [r]
        else:
            l = left(r)
            splits = split_right(right(r))
            new_non_ts = [l] + [l + '%d'%(i+1) for i in range(len(splits)-1)]

            new_rules = [x + ' -> ' + y + (new_non_ts[i+1] if i+1 < len(new_non_ts) else '') \
                 for i, (x, y) in enumerate(zip(new_non_ts, splits))]

            new_grammar.extend(new_rules)
    return compact_grammar(new_grammar)

In [71]:
for r in delete_long_rules(g):
    print(r)

S -> AB
A -> aA1
A1 -> BA2
A2 -> cB
B -> dB1
B1 -> ef


# Нормальная форма Хомского

https://neerc.ifmo.ru/wiki/index.php?title=Нормальная_форма_Хомского

Грамматикой в __нормальной форме Хомского__ (англ. Chomsky normal form) называется контекстно-свободная грамматика, в которой могут содержаться правила только следующего вида:

$A \rightarrow B C, \\
A \rightarrow a, \\
S \rightarrow \varepsilon$

, где $a$ — терминал, $A, B, C$ — нетерминалы, $S$ — стартовая вершина, $\varepsilon$ — пустая строка, стартовая вершина не содержится в правых частях правил.

Приведение грамматики к нормальной форме Хомского

Рассмотрим контекстно-свободную грамматику $\Gamma$. Для приведения ее к нормальной форме Хомского необходимо выполнить пять шагов. На каждом шаге мы строим новую $\Gamma_i$, которая допускает тот же язык, что и $\Gamma$.

1. Уберём длинные правила.
    Воспользуемся алгоритмом удаления длинных правил из грамматики. Получим грамматику $\Gamma_1$, эквивалентную исходной, содержащую правила длины 0, 1 и 2.
2. Удаление $\varepsilon$-правил.
    Воспользуемся алгоритмом удаления $\varepsilon$-правил из грамматики. Получим грамматику $\Gamma_2$, эквивалентную исходной, но в которой нет $\varepsilon$-правил.
3. Удаление цепных правил.
    Воспользуемся алгоритмом удаления цепных правил из грамматики. Алгоритм работает таким образом, что новые $\varepsilon$-правила не образуются. Получим грамматику $\Gamma_3$, эквивалентную $\Gamma$.
4. Удалим бесполезные символы.
    Воспользуемся алгоритмом удаления бесполезных символов из грамматики. Так как $\Gamma_3$ эквивалентна $\Gamma$, то бесполезные символы не могли перестать быть бесполезными. Более того, мы только удаляем правила, новые $\varepsilon$-правила и цепные правила не могли появиться.
5. Уберём ситуации, когда в правиле встречаются несколько терминалов.
    Для всех правил вида $A \rightarrow u_1 u_2$ (где $u_i$ — терминал или нетерминал) заменим все терминалы $u_i$ на новые нетерминалы $U_i$ и добавим правила $U_i \rightarrow u_i$. Теперь правила содержат либо одиночный терминал, либо строку из двух нетерминалов.

Таким образом, мы получили грамматику в нормальной форме Хомского, которая допускает тот же язык, что и $\Gamma$.

$$
S\rightarrow aXbX|aZ  \\
X\rightarrow aY|bY|\varepsilon  \\
Y\rightarrow X|cc \\
Z\rightarrow ZX \\
$$

In [72]:
g = [
    'S -> aXbX | aZ',
    'X -> aY | bY | #',
    'Y -> X | cc',
    'Z -> ZX'
]

In [75]:
g_1 = delete_long_rules(g)
# g_2 = delete_eps_generating_rules(g_1)
# g_3 = delete_unit_rules(g_2)
# g_4 = delete_useless_symbols(g_3)

In [76]:
for r in g_1:
    print(r)

S -> aS1 | aZ
S1 -> XS2
S2 -> bX
X -> aY | bY | #
Y -> X | cc
Z -> ZX


$$ S\rightarrow aS_{1}|aZ \\
X\rightarrow aY|bY|\varepsilon \\
Y\rightarrow X|cc \\
Z\rightarrow ZX \\
S_{1}\rightarrow XS_{2} \\
S_{2}\rightarrow yX $$

# Устранение непосредственной левой рекурсии

https://neerc.ifmo.ru/wiki/index.php?title=Устранение_левой_рекурсии

Говорят, что контекстно-свободная (КС) грамматика $\Gamma$ содержит __непосредственную левую рекурсию__ (англ. direct left recursion), если она содержит правило вида $A \to A\alpha$.

Методы нисходящего разбора не в состоянии работать с леворекурсивными грамматиками. Проблема в том, что продукция вида $A \Rightarrow^* A\alpha$ может применяться бесконечно долго, так и не выработав некий терминальный символ, который можно было бы сравнить со строкой. Поэтому требуется преобразование грамматики, которое бы устранило левую рекурсию.

Опишем процедуру, устраняющую все правила вида $A \to A\alpha$, для фиксированного нетерминала $A$.

1. Запишем все правила вывода из $A$ в виде:  $A \to A\alpha_1 \mid \ldots \mid A\alpha_n \mid \beta_1 \mid \ldots \mid \beta_m$, где
    * $\alpha$ — непустая последовательность терминалов и нетерминалов ($\alpha \nrightarrow \varepsilon$);
    * $\beta$ — непустая последовательность терминалов и нетерминалов, не начинающаяся с $A$.
    
2. Заменим правила вывода из $A$ на $A \to\beta_1A^\prime \mid \ldots\ \mid \beta_mA^\prime \mid \beta_1 \mid \ldots \mid \beta_m$.
3. Создадим новый нетерминал ${A^\prime} \to \alpha_1{A^\prime} \mid \ldots \mid \alpha_n{A^\prime} \mid \alpha_1 \mid \ldots \mid \alpha_n$.

Изначально нетерминал $A$ порождает строки вида $\beta\alpha_{i0}\alpha_{i1} \ldots \alpha_{ik}$. В новой грамматике нетерминал $A$ порождает $\beta{A^\prime}$, а $A^\prime$ порождает строки вида $\alpha_{i0}\alpha_{i1} \ldots \alpha_{ik}$. Из этого очевидно, что изначальная грамматика эквивалентна новой.